Connect Drive

In [1]:
from google.colab import drive 
drive.mount('/content/drive')

Mounted at /content/drive


Install requirements

In [2]:
%%capture
!pip install kaggle
!pip install jiwer 
!pip install datasets
!pip install transformers 
!pip install wandb

Check GPU info

In [3]:
gpu_info = !nvidia-smi
gpu_info = '\n'.join(gpu_info)
if gpu_info.find('failed') >= 0:
  print('Not connected to a GPU')
else:
  print(gpu_info)

Not connected to a GPU


Prepare to download dataset from kaggle

In [4]:
!mkdir ~/.kaggle 
!cp /content/drive/MyDrive/kaggle.json ~/.kaggle/kaggle.json 
!chmod 600 ~/.kaggle/kaggle.json

Download Dataset

In [5]:
!kaggle datasets download anishshilpakar/cleaned-asr-data

100% 4.09G/4.10G [02:09<00:00, 47.3MB/s]
100% 4.10G/4.10G [02:09<00:00, 34.0MB/s]


Unzip Dataset

In [6]:
%%capture
!unzip cleaned-asr-data.zip

Imports

In [7]:
from datasets import Audio, ClassLabel, load_dataset, load_metric
from IPython.display import display, HTML
from sklearn.model_selection import train_test_split
from torch._C import device
from transformers import (
    EarlyStoppingCallback,
    Trainer,
    TrainingArguments,
    Wav2Vec2CTCTokenizer,
    Wav2Vec2FeatureExtractor,
    Wav2Vec2ForCTC,
    Wav2Vec2Processor,
    trainer_utils
)

In [8]:
import json
import os
from dataclasses import dataclass, field
from typing import Any, Dict, List, Optional, Union

import IPython.display as ipd
import jiwer
import librosa
import numpy as np
import pandas as pd
import random
import re
import torch
import torch.functional as F
import torch.nn as nn
import torch.optim as optim
import torchaudio
import wandb

Connect to weight and bias

In [9]:
wandb.login(key="f8309bfa7d3b1d6d877d79040b6d30b6dc26afcb") # add wandb API key
wandb.init(project="speech")

wandb: W&B API key is configured. Use `wandb login --relogin` to force relogin
wandb: WARNING If you're specifying your api key in code, ensure this code is not shared publicly.
wandb: WARNING Consider setting the WANDB_API_KEY environment variable, or running `wandb login` from the command line.
wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc
wandb: Currently logged in as: anish_shilpakar. Use `wandb login --relogin` to force relogin


Connect to huggingface to save models

In [10]:
from huggingface_hub import notebook_login

notebook_login()

Login successful
Your token has been saved to /root/.huggingface/token


In [11]:
%%capture
!apt install git-lfs

In [12]:
project_folder_location = '/content/drive/MyDrive/major_project/wav2vec'

In [13]:
%cd {project_folder_location}

/content/drive/MyDrive/major_project/wav2vec


In [17]:
# !mkdir csv_files
# !mkdir trained_models
# !mkdir checkpoints
# !mkdir tokenizer
# !mkdir processor 

In [14]:
csv_folder_path = '/content/drive/MyDrive/major_project/wav2vec/csv_files'
trained_model_path = '/content/drive/MyDrive/major_project/wav2vec/trained_model_wav2vec_0.3_dropout'
processor_path = '/content/drive/MyDrive/major_project/wav2vec/processor_0.3dropout'
checkpoints_path = '/content/drive/MyDrive/major_project/wav2vec/checkpoints/'
tokenizer_path = '/content/drive/MyDrive/major_project/wav2vec/tokenizer/'

In [15]:
colnames = ["path","labels"]
audio_path = '/content/data/data/audio/'
transcript_path = '/content/transcript_durations/dataset_duration_bt_5_and_10.csv'
df = pd.read_csv(transcript_path,usecols=colnames)
df["path"] = audio_path + df["path"] + ".flac"
print(df.shape)
df.head()

(3231, 2)


,path,labels
0,/content/data/data/audio/nep_0546_1535070415.flac,दामोदर कुण्ड नेपालको पश्चिमाञ्चल विकास क्षेत्र...
1,/content/data/data/audio/nep_2099_2296423149.flac,ब्राजिलको गाउँमा बस्नेमारिया आफ्नो भाग्य खोज्द...
2,/content/data/data/audio/nep_0546_0685084826.flac,बर्दिया राष्ट्रिय निकुन्ज पूर्वमा चेपाङ नदी पश...
3,/content/data/data/audio/nep_3960_6138806624.flac,काठमाडौँका बासिन्दा इन्द्रजात्राको रमाइलोमा रम...
4,/content/data/data/audio/nep_0546_3782097543.flac,कपिलवस्तु जिल्लाको उत्तरी क्षेत्रमा महेन्द्र र...


In [16]:
TEST_RATIO = 0.90 
VAL_RATIO = 0.75

In [17]:
train_df, test_df = train_test_split(df, random_state = 0, train_size = TEST_RATIO)
train_df, val_df = train_test_split(train_df, random_state = 0, train_size = VAL_RATIO)

In [18]:
train_df.to_csv(csv_folder_path+'/train.csv',index=False)
val_df.to_csv(csv_folder_path+'/val.csv',index=False)
test_df.to_csv(csv_folder_path+'/test.csv',index=False)
!ls {csv_folder_path}

test.csv  train.csv  val.csv


In [19]:
data_files = {
    "train": "train.csv",
    "validation": "val.csv",
    "test": "test.csv"
}

In [20]:
train_data = load_dataset(csv_folder_path, data_files = data_files, split = "train")
val_data = load_dataset(csv_folder_path, data_files = data_files, split = "validation")

Extracting data files:   0%|          | 0/3 [00:00<?, ?it/s]

0 tables [00:00, ? tables/s]

0 tables [00:00, ? tables/s]

0 tables [00:00, ? tables/s]

Dataset csv downloaded and prepared to /root/.cache/huggingface/datasets/csv/csv_files-4fa672eb7fcbed21/0.0.0/6b34fb8fcf56f7c8ba51dc895bfa2bfbe43546f190a60fcf74bb5e8afdcc2317. Subsequent calls will reuse this data.


In [21]:
def show_random_elements(dataset, num_examples=10):
    assert num_examples <= len(dataset), "Can't pick more elements than there are in the dataset."
    picks = []
    for _ in range(num_examples):
        pick = random.randint(0, len(dataset)-1)
        while pick in picks:
            pick = random.randint(0, len(dataset)-1)
        picks.append(pick)
    
    df = pd.DataFrame(dataset[picks])
    display(HTML(df.to_html()))

In [22]:
show_random_elements(train_data.remove_columns(["path"]), num_examples = 5)

,labels
0,लोकनायक जयप्रकाश नारायण
1,तर्फबाट वार्सेल्सको सन्धिमा
2,साधनभूत वहाँको व्यापक
3,वेखसिम्ले सिम्थली चौवास
4,मिथकीय स्वैरकल्पनाको


In [23]:
chars_to_ignore = '[\,\?\.\!\-\;\:\"\“\%\‘\”\�]'

def remove_special_characters(batch):
    batch["labels"] = re.sub(chars_to_ignore, '', batch["labels"]).lower() + " "
    return batch

In [24]:
train_data = train_data.map(remove_special_characters)
val_data = val_data.map(remove_special_characters)

  0%|          | 0/2180 [00:00<?, ?ex/s]

  0%|          | 0/727 [00:00<?, ?ex/s]

In [25]:
vocab_path = '/content/data/data/vocabulary/vocab.json'
tokenizer = Wav2Vec2CTCTokenizer(
    vocab_path, 
    unk_token = "[UNK]", 
    pad_token = "[PAD]", 
    word_delimiter_token = "|"
)

In [46]:
repo_name = "asr1"

In [47]:
tokenizer.push_to_hub(repo_name)

tokenizer config file saved in /tmp/tmp31d1gork/tokenizer_config.json
Special tokens file saved in /tmp/tmp31d1gork/special_tokens_map.json
Uploading the following files to anish-shilpakar/asr1: vocab.json,tokenizer_config.json,special_tokens_map.json


CommitInfo(commit_url='https://huggingface.co/anish-shilpakar/asr1/commit/6e7164e81bf6ffca984e676dd8a304a55df01cbf', commit_message='Upload tokenizer', commit_description='', oid='6e7164e81bf6ffca984e676dd8a304a55df01cbf', pr_url=None, pr_revision=None, pr_num=None)

In [28]:
tokenizer.save_pretrained(tokenizer_path)

('/content/drive/MyDrive/major_project/wav2vec/tokenizer/tokenizer_config.json',
 '/content/drive/MyDrive/major_project/wav2vec/tokenizer/special_tokens_map.json',
 '/content/drive/MyDrive/major_project/wav2vec/tokenizer/vocab.json',
 '/content/drive/MyDrive/major_project/wav2vec/tokenizer/added_tokens.json')

In [29]:
feature_extractor = Wav2Vec2FeatureExtractor(
    feature_size = 1, 
    sampling_rate = 16000, 
    padding_value = 0.0, 
    do_normalize = True, 
    return_attention_mask = True
)

In [30]:
processor = Wav2Vec2Processor(
    feature_extractor = feature_extractor, 
    tokenizer = tokenizer
)

In [48]:
processor.push_to_hub(repo_name)

Feature extractor saved in /tmp/tmphssrw0a4/preprocessor_config.json
tokenizer config file saved in /tmp/tmphssrw0a4/tokenizer_config.json
Special tokens file saved in /tmp/tmphssrw0a4/special_tokens_map.json
Uploading the following files to anish-shilpakar/asr1: vocab.json,tokenizer_config.json,special_tokens_map.json,preprocessor_config.json


CommitInfo(commit_url='https://huggingface.co/anish-shilpakar/asr1/commit/59b9d0b242a71b9d196a6f1c099e20b228f2f560', commit_message='Upload processor', commit_description='', oid='59b9d0b242a71b9d196a6f1c099e20b228f2f560', pr_url=None, pr_revision=None, pr_num=None)

In [32]:
processor.save_pretrained(processor_path)

In [33]:
train_data

Dataset({
    features: ['path', 'labels'],
    num_rows: 2180
})

In [34]:
def speech_file_to_array_fn(batch):
    speech_array, sampling_rate = torchaudio.load(batch["path"])
    batch["speech"] = speech_array[0].numpy()
    batch["sampling_rate"] = sampling_rate
    batch["target_text"] = batch["labels"]
    
    resampler = torchaudio.transforms.Resample(sampling_rate, 16000)
    batch["speech"] = resampler(speech_array).squeeze().numpy()
    batch["sampling_rate"] = 16000
    
    return batch

In [35]:
train_data = train_data.map(speech_file_to_array_fn, remove_columns=train_data.column_names)
val_data = val_data.map(speech_file_to_array_fn, remove_columns=val_data.column_names)

  0%|          | 0/2180 [00:00<?, ?ex/s]

  0%|          | 0/727 [00:00<?, ?ex/s]

In [36]:
def prepare_dataset(batch, processor):
        
    # check that all files have the correct sampling rate
    assert (
        len(set(batch["sampling_rate"])) == 1
    ), f"Make sure all inputs have the same sampling rate of {processor.feature_extractor.sampling_rate}."

    batch["input_values"] = processor(batch["speech"], sampling_rate = batch["sampling_rate"][0]).input_values
    
    with processor.as_target_processor():
        batch["labels"] = processor(batch["target_text"]).input_ids
    return batch

In [37]:
train_data = train_data.map(prepare_dataset, fn_kwargs = {"processor": processor}, remove_columns = train_data.column_names, batch_size = 8, num_proc = 4, batched = True)
val_data = val_data.map(prepare_dataset, fn_kwargs = {"processor": processor}, remove_columns = val_data.column_names, batch_size = 8, num_proc = 4, batched = True)

#0:   0%|          | 0/69 [00:00<?, ?ba/s]

#1:   0%|          | 0/69 [00:00<?, ?ba/s]

#2:   0%|          | 0/69 [00:00<?, ?ba/s]

#3:   0%|          | 0/69 [00:00<?, ?ba/s]

/usr/local/lib/python3.7/dist-packages/transformers/feature_extraction_utils.py:162: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  tensor = as_tensor(value)
/usr/local/lib/python3.7/dist-packages/transformers/models/wav2vec2/processing_wav2vec2.py:155: UserWarning: `as_target_processor` is deprecated and will be removed in v5 of Transformers. You can process your labels by using the argument `text` of the regular `__call__` method (either in the same call as your audio inputs, or in a separate call.
  "`as_target_processor` is deprecated and will be removed in v5 of Transformers. You can process your "
/usr/local/lib/python3.7/dist-packages/transformers/feature_extraction_utils.py:162: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which 

#1:   0%|          | 0/23 [00:00<?, ?ba/s]

#0:   0%|          | 0/23 [00:00<?, ?ba/s]

#2:   0%|          | 0/23 [00:00<?, ?ba/s]

#3:   0%|          | 0/23 [00:00<?, ?ba/s]

/usr/local/lib/python3.7/dist-packages/transformers/feature_extraction_utils.py:162: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  tensor = as_tensor(value)
/usr/local/lib/python3.7/dist-packages/transformers/models/wav2vec2/processing_wav2vec2.py:155: UserWarning: `as_target_processor` is deprecated and will be removed in v5 of Transformers. You can process your labels by using the argument `text` of the regular `__call__` method (either in the same call as your audio inputs, or in a separate call.
  "`as_target_processor` is deprecated and will be removed in v5 of Transformers. You can process your "
/usr/local/lib/python3.7/dist-packages/transformers/feature_extraction_utils.py:162: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which 

In [38]:
@dataclass
class DataCollatorCTCWithPadding:
    """
    Data collator that will dynamically pad the inputs received.
    Args:
        processor (:class:`~transformers.Wav2Vec2Processor`)
            The processor used for proccessing the data.
        padding (:obj:`bool`, :obj:`str` or :class:`~transformers.tokenization_utils_base.PaddingStrategy`, `optional`, defaults to :obj:`True`):
            Select a strategy to pad the returned sequences (according to the model's padding side and padding index)
            among:
            * :obj:`True` or :obj:`'longest'`: Pad to the longest sequence in the batch (or no padding if only a single
              sequence if provided).
            * :obj:`'max_length'`: Pad to a maximum length specified with the argument :obj:`max_length` or to the
              maximum acceptable input length for the model if that argument is not provided.
            * :obj:`False` or :obj:`'do_not_pad'` (default): No padding (i.e., can output a batch with sequences of
              different lengths).
        max_length (:obj:`int`, `optional`):
            Maximum length of the ``input_values`` of the returned list and optionally padding length (see above).
        max_length_labels (:obj:`int`, `optional`):
            Maximum length of the ``labels`` returned list and optionally padding length (see above).
        pad_to_multiple_of (:obj:`int`, `optional`):
            If set will pad the sequence to a multiple of the provided value.
            This is especially useful to enable the use of Tensor Cores on NVIDIA hardware with compute capability >=
            7.5 (Volta).
    """

    processor: Wav2Vec2Processor
    padding: Union[bool, str] = True
    max_length: Optional[int] = None
    max_length_labels: Optional[int] = None
    pad_to_multiple_of: Optional[int] = None
    pad_to_multiple_of_labels: Optional[int] = None

    def __call__(self, features: List[Dict[str, Union[List[int], torch.Tensor]]]) -> Dict[str, torch.Tensor]:
        # split inputs and labels since they have to be of different lenghts and need
        # different padding methods
        input_features = [{"input_values": feature["input_values"]} for feature in features]
        label_features = [{"input_ids": feature["labels"]} for feature in features]

        batch = self.processor.pad(
            input_features,
            padding=self.padding,
            max_length=self.max_length,
            pad_to_multiple_of=self.pad_to_multiple_of,
            return_tensors="pt",
        )
        with self.processor.as_target_processor():
            labels_batch = self.processor.pad(
                label_features,
                padding=self.padding,
                max_length=self.max_length_labels,
                pad_to_multiple_of=self.pad_to_multiple_of_labels,
                return_tensors="pt",
            )

        # replace padding with -100 to ignore loss correctly
        labels = labels_batch["input_ids"].masked_fill(labels_batch.attention_mask.ne(1), -100)

        batch["labels"] = labels

        return batch

In [39]:
data_collator = DataCollatorCTCWithPadding(processor = processor, padding = True)

In [40]:
wer_metric = load_metric("wer")
cer_metric = load_metric("cer", revision = "master")

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:1: FutureWarning: load_metric is deprecated and will be removed in the next major version of datasets. Use 'evaluate.load' instead, from the new library 🤗 Evaluate: https://huggingface.co/docs/evaluate
  """Entry point for launching an IPython kernel.


In [41]:
def compute_metrics(pred):
    pred_logits = pred.predictions
    pred_ids = np.argmax(pred_logits, axis=-1)

    pred.label_ids[pred.label_ids == -100] = processor.tokenizer.pad_token_id

    pred_str = processor.batch_decode(pred_ids)
    # we do not want to group tokens when computing the metrics
    label_str = processor.batch_decode(pred.label_ids, group_tokens=False)

    wer = wer_metric.compute(predictions=pred_str, references=label_str)
    cer = cer_metric.compute(predictions=pred_str, references=label_str)
    
    # cer_best = 1 - cer because load best model is considering greater value of cer for better results
    accuracy = 1 - cer
    # wandb.log({'wer':wer, 'cer':cer})
    # experiment.log_metric('wer', wer)
    # experiment.log_metric('cer', cer)
    
    return {"wer": wer, "cer": cer, "accuracy": accuracy}

In [42]:
model = Wav2Vec2ForCTC.from_pretrained(
    "facebook/wav2vec2-large-xlsr-53", 
    attention_dropout = 0.3,
    hidden_dropout = 0.3,
    feat_proj_dropout = 0.0,
    mask_time_prob = 0.05,
    layerdrop = 0.1,
    gradient_checkpointing = True, 
    ctc_loss_reduction = "mean", 
    pad_token_id = processor.tokenizer.pad_token_id,
    vocab_size = len(processor.tokenizer)
)

Downloading:   0%|          | 0.00/1.77k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/1.27G [00:00<?, ?B/s]

Some weights of the model checkpoint at facebook/wav2vec2-large-xlsr-53 were not used when initializing Wav2Vec2ForCTC: ['quantizer.weight_proj.bias', 'project_q.weight', 'quantizer.weight_proj.weight', 'project_q.bias', 'project_hid.weight', 'quantizer.codevectors', 'project_hid.bias']
- This IS expected if you are initializing Wav2Vec2ForCTC from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing Wav2Vec2ForCTC from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of Wav2Vec2ForCTC were not initialized from the model checkpoint at facebook/wav2vec2-large-xlsr-53 and are newly initialized: ['lm_head.weight', 'lm_head.bias']
You should probably TRAIN this model on a down-stream task to be able to u

In [43]:
model.freeze_feature_encoder()

In [53]:
%cd /content/

cleaned-asr-data.zip  data  drive  sample_data	transcript_durations  wandb


In [54]:
training_args = TrainingArguments(
  output_dir = repo_name,
  group_by_length=True,
  per_device_train_batch_size=16,
  gradient_accumulation_steps=2,
  evaluation_strategy="steps",
  num_train_epochs=30,
  fp16 = True,
  save_steps = 100,
  eval_steps=100,
  logging_steps=100,
  learning_rate=3e-4,
  warmup_steps=180,
  save_total_limit = 2,
  overwrite_output_dir=True,
  gradient_checkpointing = True,
  report_to = "wandb",
  metric_for_best_model = 'accuracy',
  load_best_model_at_end=True,
  push_to_hub=True,
  hub_model_id="anish-shilpakar/asr1"
)

PyTorch: setting up devices


In [55]:
trainer = Trainer(
    model = model,
    data_collator = data_collator,
    args = training_args,
    compute_metrics = compute_metrics,
    train_dataset = train_data,
    eval_dataset = val_data,
    tokenizer = processor.feature_extractor,
)

Cloning https://huggingface.co/anish-shilpakar/asr1 into local empty directory.
Using cuda_amp half precision backend


In [51]:
trainer.train()

/usr/local/lib/python3.7/dist-packages/transformers/optimization.py:310: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  FutureWarning,
***** Running training *****
  Num examples = 2180
  Num Epochs = 30
  Instantaneous batch size per device = 16
  Total train batch size (w. parallel, distributed & accumulation) = 32
  Gradient Accumulation steps = 2
  Total optimization steps = 2040
Automatic Weights & Biases logging enabled, to disable set os.environ["WANDB_DISABLED"] = "true"
/usr/local/lib/python3.7/dist-packages/transformers/models/wav2vec2/processing_wav2vec2.py:155: UserWarning: `as_target_processor` is deprecated and will be removed in v5 of Transformers. You can process your labels by using the argument `text` of the regular `__call__` method (either in the same call as your audio inputs, or in a separate c

Step,Training Loss,Validation Loss,Wer,Cer,Accuracy
50,218.215000,42.558418,1.000000,1.019401,-0.019401
100,81.923600,42.557514,1.000000,1.019401,-0.019401
150,98.546800,42.556053,1.000000,1.019401,-0.019401
200,251.541400,nan,1.000000,0.996492,0.003508


***** Running Evaluation *****
  Num examples = 727
  Batch size = 8
Saving model checkpoint to asr1/checkpoint-50
Configuration saved in asr1/checkpoint-50/config.json
Model weights saved in asr1/checkpoint-50/pytorch_model.bin
Feature extractor saved in asr1/checkpoint-50/preprocessor_config.json
Feature extractor saved in asr1/preprocessor_config.json
/usr/local/lib/python3.7/dist-packages/transformers/models/wav2vec2/processing_wav2vec2.py:155: UserWarning: `as_target_processor` is deprecated and will be removed in v5 of Transformers. You can process your labels by using the argument `text` of the regular `__call__` method (either in the same call as your audio inputs, or in a separate call.
  "`as_target_processor` is deprecated and will be removed in v5 of Transformers. You can process your "
***** Running Evaluation *****
  Num examples = 727
  Batch size = 8
Saving model checkpoint to asr1/checkpoint-100
Configuration saved in asr1/checkpoint-100/config.json
Model weights saved

KeyboardInterrupt: ignored

In [105]:
trainer.push_to_hub("anish-shilpakar/nepali-wav2vec-v1")

OSError: ignored

In [87]:
trainer.save_model(trained_model_path)

Saving model checkpoint to /content/drive/MyDrive/major_project/wav2vec/trained_models/
Configuration saved in /content/drive/MyDrive/major_project/wav2vec/trained_models/config.json
Model weights saved in /content/drive/MyDrive/major_project/wav2vec/trained_models/pytorch_model.bin
Feature extractor saved in /content/drive/MyDrive/major_project/wav2vec/trained_models/preprocessor_config.json


TESTING

In [116]:
model = Wav2Vec2ForCTC.from_pretrained("/content/drive/MyDrive/major_project/wav2vec/trained_model_wav2vec_4_5_0.1dropout").to("cuda")
processor = Wav2Vec2Processor.from_pretrained("/content/drive/MyDrive/major_project/wav2vec/processor_wav2vec_0.1_dropout_4_5")

loading configuration file /content/drive/MyDrive/major_project/wav2vec/trained_model_wav2vec_4_5_0.1dropout/config.json
Model config Wav2Vec2Config {
  "_name_or_path": "facebook/wav2vec2-large-xlsr-53",
  "activation_dropout": 0.0,
  "adapter_kernel_size": 3,
  "adapter_stride": 2,
  "add_adapter": false,
  "apply_spec_augment": true,
  "architectures": [
    "Wav2Vec2ForCTC"
  ],
  "attention_dropout": 0.1,
  "bos_token_id": 1,
  "classifier_proj_size": 256,
  "codevector_dim": 768,
  "contrastive_logits_temperature": 0.1,
  "conv_bias": true,
  "conv_dim": [
    512,
    512,
    512,
    512,
    512,
    512,
    512
  ],
  "conv_kernel": [
    10,
    3,
    3,
    3,
    3,
    2,
    2
  ],
  "conv_stride": [
    5,
    2,
    2,
    2,
    2,
    2,
    2
  ],
  "ctc_loss_reduction": "mean",
  "ctc_zero_infinity": false,
  "diversity_loss_weight": 0.1,
  "do_stable_layer_norm": true,
  "eos_token_id": 2,
  "feat_extract_activation": "gelu",
  "feat_extract_dropout": 0.0,
  "f

RuntimeError: ignored

In [114]:
import gc 
del model
gc.collect()

108

In [115]:
torch.cuda.empty_cache()

In [89]:
def segmentLargeArray(inputTensor,chunksize=200000):
    # print(inputTensor)
    list_of_segments = []
    tensor_length = inputTensor.shape[1]
    for i in range(0,tensor_length+1,chunksize):
        list_of_segments.append(inputTensor[:,i:i+chunksize])
    return list_of_segments 

In [90]:
def predict_from_speech(file):
    speech_array, sampling_rate = torchaudio.load(file)
    # print(speech_array,sampling_rate)
    resampler = torchaudio.transforms.Resample(sampling_rate, 16000)
    resampled_array = resampler(speech_array).squeeze()
    if len(resampled_array.shape) == 1:
        resampled_array = resampled_array.reshape([1,resampled_array.shape[0]])
    # print(resampled_array.shape[1])
    if resampled_array.shape[1] >= 200000:
        print('The input file is longer than 10 seconds')
        list_of_segments = segmentLargeArray(resampled_array)
        # print(list_of_segments)
        output = ''
        for segment in list_of_segments:
            logits = model(segment.to("cuda")).logits
            pred_ids = torch.argmax(logits,dim=-1)[0]
            output += processor.decode(pred_ids)
        print(f"Prediction:\n{output}")
    else:
        print('The input file is less than 10 seconds')
        logits = model(resampled_array.to("cuda")).logits
        # print(logits)
        pred_ids = torch.argmax(logits, dim = -1)[0]
        print("Prediction:")
        print(processor.decode(pred_ids))

In [91]:
test1 = '/content/drive/MyDrive/major_project/test/rec2.wav'
test2 = '/content/drive/MyDrive/major_project/test/testy.flac'

In [110]:
# predict_from_speech(test_audio_ip1)
predict_from_speech(test1)
predict_from_speech(test2)

The input file is less than 10 seconds
Prediction:
नेपाल एउटा सुन्डर देशहो
The input file is longer than 10 seconds


RuntimeError: ignored